In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import time

## Goal: to optimize the Sharpe ratio on historical data (206 stocks)

### Constraints: 
1. Sum of all weights is 100%
3. The weight of each stock can be 1-10% or 0% (this is necessary for the convenience of working with the portfolio, to avoid securities with very small weights)

#### Data: 
daily returns of 206 stocks (adjusted for dividends and splits; source: MOEX) 

In [2]:
ret = pd.read_csv('../rets.csv').drop("date", axis=1)

In [3]:
hist_cov = ret.cov()
hist_cov

,+МосЭнерго,ETLN-гдр,FIXP-гдр,OKEY-гдр,OZON-адр,VEON,iАРТГЕН ао,iАвиастКао,iВУШХолднг,iГЕНЕТИКО,...,ЭсЭфАй ао,ЮГК,ЮТэйр ао,ЮУНК ао,ЮжКузб. ао,Юнипро ао,ЯТЭК ао,Яковлев-3,Якутскэн-п,Якутскэнрг
+МосЭнерго,0.000397,0.000238,0.000230,0.000220,0.000258,0.000246,0.000230,0.000302,0.000262,0.000259,...,0.000197,0.000219,0.000250,0.000198,0.000137,0.000246,0.000219,0.000350,0.000205,0.000230
ETLN-гдр,0.000238,0.000773,0.000353,0.000342,0.000455,0.000388,0.000400,0.000458,0.000438,0.000359,...,0.000400,0.000362,0.000381,0.000364,0.000172,0.000302,0.000384,0.000504,0.000281,0.000306
FIXP-гдр,0.000230,0.000353,0.000793,0.000272,0.000364,0.000319,0.000382,0.000344,0.000340,0.000350,...,0.000340,0.000236,0.000357,0.000296,0.000149,0.000297,0.000275,0.000416,0.000231,0.000238
OKEY-гдр,0.000220,0.000342,0.000272,0.000889,0.000300,0.000278,0.000178,0.000357,0.000320,0.000222,...,0.000282,0.000215,0.000362,0.000229,0.000165,0.000268,0.000301,0.000358,0.000204,0.000211
OZON-адр,0.000258,0.000455,0.000364,0.000300,0.000636,0.000349,0.000394,0.000450,0.000417,0.000352,...,0.000437,0.000343,0.000325,0.000333,0.000182,0.000333,0.000328,0.000498,0.000266,0.000329
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Юнипро ао,0.000246,0.000302,0.000297,0.000268,0.000333,0.000349,0.000267,0.000365,0.000299,0.000282,...,0.000333,0.000189,0.000284,0.000250,0.000145,0.000467,0.000276,0.000425,0.000213,0.000223
ЯТЭК ао,0.000219,0.000384,0.000275,0.000301,0.000328,0.000276,0.000335,0.000414,0.000336,0.000351,...,0.000352,0.000284,0.000300,0.000312,0.000205,0.000276,0.000715,0.000496,0.000242,0.000321
Яковлев-3,0.000350,0.000504,0.000416,0.000358,0.000498,0.000381,0.000554,0.000829,0.000597,0.000491,...,0.000466,0.000278,0.000470,0.000353,0.000222,0.000425,0.000496,0.001140,0.000330,0.000373
Якутскэн-п,0.000205,0.000281,0.000231,0.000204,0.000266,0.000202,0.000231,0.000302,0.000258,0.000265,...,0.000240,0.000201,0.000225,0.000247,0.000157,0.000213,0.000242,0.000330,0.000501,0.000412


Since it is not possible to optimize directly under the constraints above, we use a two-step approach: 
1. In the first step, we optimize the weights with a constraint of 0-10%. 
2. In the second step, we drop assets with a weight of <1% and redistribute the weight of those dropped across the remaining assets using an optimizer.

In [4]:
time_start = time.time()
ret_full = ret.copy(deep=True)
def sharpe_func(weights):
    hist_mean = ret.mean(axis=0).to_frame()
    hist_cov = ret.cov()

    port_ret = np.dot(weights.T, hist_mean.values)
    port_std = np.sqrt(np.dot(weights.T, np.dot(hist_cov, weights)))
    return -1 * port_ret/port_std

def weight_cons(weights):
    return np.sum(weights) - 1

constraint = [{'type': 'eq', 'fun': weight_cons}]

bounds_lim = [(0.,0.1)]*len(ret.columns)

init = [1/len(ret.columns)]*len(ret.columns)

optimal = minimize(fun=sharpe_func,
                   x0=init,
                   bounds=bounds_lim,
                   constraints=constraint,
                   method='SLSQP',
                   tol=1e-18
				)

optimal_x = optimal['x']
first_iteration_weights = pd.DataFrame(data=list(zip(ret.columns, optimal_x)), columns=['stock', 'weight'])
first_iteration_weights = first_iteration_weights[first_iteration_weights['weight']>=0.01]
print('Weight after 1st iteration = ',first_iteration_weights['weight'].sum() )
out = pd.DataFrame(data=list(zip(ret.columns, optimal_x)), columns=['st', 'val'])
weight_after_first_iteration = first_iteration_weights['weight'].sum()

if first_iteration_weights['weight'].sum() < 1-1e-12:
    print('Initialize 2nd part!')
    ret_second_iteration = ret_full.copy(deep=True)[first_iteration_weights['stock'].tolist()]
    initial_weight_for_2_iter = dict(zip(first_iteration_weights['stock'].tolist(),first_iteration_weights['weight'].tolist()))
    
    
    '''
		SECOND PART: we will launch the second optimizer to change the weight boundaries from 0-10% to 1-10%
	'''
    
    
    
    def sharpe_func_2(weights):
        hist_mean = ret_second_iteration.mean(axis=0).to_frame()
        hist_cov = ret_second_iteration.cov()
        initial_weights = np.array([initial_weight_for_2_iter[st_i] for st_i in ret_second_iteration.columns])
        weights_final = initial_weights+weights
        port_ret = np.dot(weights_final.T, hist_mean.values)
        port_std = np.sqrt(np.dot(weights_final.T, np.dot(hist_cov, weights_final)))
        return -1 * port_ret / port_std
    
    def weight_cons(weights):
        return np.sum(weights) - 1 - weight_after_first_iteration 
    
    constraint = [{'type': 'eq', 'fun': weight_cons}]
    init = [0]*len(ret_second_iteration.columns)
    
    bounds_lim = [(0.,max(0.,0.1-initial_weight_for_2_iter[st_i]) ) for st_i in ret_second_iteration.columns ]

    optimal = minimize(fun=sharpe_func_2,
                       x0=init,
                       bounds=bounds_lim,
                       constraints=constraint,
                       method='SLSQP',
                       tol=1e-18
					)
    
    optimal_x = np.array([initial_weight_for_2_iter[x] for x in ret_second_iteration.columns])+optimal['x']

print(f'Time taken = {round(time.time() - time_start,2)} sec'   )
optimal_weights = pd.DataFrame(data=zip(ret.columns,optimal_x),columns=['stock','weight']).sort_values(by='weight',ascending=False)
print(f'Optimal sharpe value = {-optimal.fun}')

Weight after 1st iteration =  0.9968338740828434
Initialize 2nd part!
Time taken = 3.56 sec
Optimal sharpe value = 0.1252718386451649


# MCS approach (test results of the Rust code)

In [5]:
weights_MCS = np.array(
	 [
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.10000000000000017,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.07705629905954528,
        0.0,
        0.0,
        0.0,
        0.0,
        0.10000000000000017,
        0.10000000000000017,
        0.10000000000000017,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.022943700940452175,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.1,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.10000000000000017,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.10000000000000017,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.10000000000000017,
        0.10000000000000017,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
    ],
)

In [6]:
sharpe_func(weights_MCS)

array([-0.12559579])

In [7]:
weights_MCS.sum(), weights_MCS[weights_MCS!=0.0].min(), weights_MCS.max()

(np.float64(0.9999999999999989),
 np.float64(0.022943700940452175),
 np.float64(0.10000000000000017))